# Sjekk lengder, NVDB rapporter

Tar data for et område og teller lengde ihht NVDB rapporter regelverk

In [1]:
import STARTHER 
import pandas as pd
import nvdbapiv3


Legger NVDB api til søkestien


Laster data 

In [2]:
f = nvdbapiv3.nvdbFagdata( 845 )
f.filter({ 'kontraktsomrade' : '9305 Sunnfjord'})
ff = pd.DataFrame( f.to_records() )

In [3]:
ff

,objekttype,nvdbId,versjon,startdato,veglenkesekvensid,detaljnivå,typeVeg,kommune,fylke,vref,...,Lengde,Høyde,Gjennomsnittlig avstand fra veg,Adkomst,Etableringsår,Produsent,Eier,Vedlikeholdsansvarlig,Styrke,Helningsvinkel gjerde
0,845,642570737,1,2015-10-27,384520,Vegtrase og kjørebane,Gang- og sykkelveg,4640,46,RV13 S47D200 m397-498,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,845,642570738,1,2015-10-27,384520,Vegtrase og kjørebane,Gang- og sykkelveg,4640,46,RV13 S47D200 m291-399,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,845,642570755,1,2015-10-27,1724813,Vegtrase og kjørebane,Enkel bilveg,4640,46,RV13 S47D1 m2269-2306,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,845,642570756,1,2015-10-27,1724816,Vegtrase og kjørebane,Enkel bilveg,4640,46,RV13 S47D1 m2216-2262,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,845,642570756,1,2015-10-27,1724813,Vegtrase og kjørebane,Enkel bilveg,4640,46,RV13 S47D1 m2262-2267,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,845,643232564,1,2015-10-30,384520,Vegtrase og kjørebane,Gang- og sykkelveg,4640,46,RV13 S47D200 m1331-1408,...,77.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,845,643232565,1,2015-10-30,384520,Vegtrase og kjørebane,Gang- og sykkelveg,4640,46,RV13 S47D200 m1227-1300,...,73.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,845,844165080,1,2017-11-27,384519,Vegtrase og kjørebane,Gang- og sykkelveg,4640,46,RV13 S48D200 m925-962,...,40.1,4.0,79.0,Til fots,2017.0,Geobrügg,"Stat, Statens vegvesen",Statens vegvesen,2000.0,NaN
8,845,916900140,1,2019-02-22,384023,Vegtrase og kjørebane,Enkel bilveg,4647,46,EV39 S55D1 m2918-3113,...,207.0,4.0,60.0,Til fots,NaN,NaN,NaN,NaN,3000.0,60.0
9,845,916900141,1,2019-02-22,2631223,Vegtrase og kjørebane,Enkel bilveg,4647,46,EV39 S61D1 m6767-6881,...,114.0,5.0,45.0,Til fots,NaN,NaN,NaN,NaN,3000.0,62.0


Finner riktig antall per vegkategori og trafikantgruppe. Merk at det ikke gjør noen forskjell om vi dropper _trafikantgruppe_ fra oppsummeringen nedenfor; jeg har sjekket at i dette tilfellet har vi ingen objekt som er stedfestet på begge trafikantgruppene samtidig. Dette kan være en interessant snublefelle i noen tilfeller, men ikke her. 

In [14]:
ff.drop_duplicates( ['nvdbId', 'trafikantgruppe']).groupby( [ 'vegkategori', 'trafikantgruppe' ] ).agg( { 'nvdbId' : 'count'})

nvdbId
vegkategori trafikantgruppe        
E           K                     6
R           G                     6
            K                    10

Deler datasettet i to, basert på om vi har "Lengde"-egenskap eller ikke. Hvis vi har lengde-egenskap må vi slette duplikater, slik at et objekt ikke telles flere ganger. Hvis ikke skal vi telle lengden av objektets stedfesting. Etterpå spleiser vi dem igjen

In [4]:
f_egenskap = ff[ ~ff['Lengde'].isnull()].drop_duplicates( 'nvdbId').copy()

f_veg = ff[ ff['Lengde'].isnull() ].copy()
f_veg['Lengde'] = f_veg['segmentlengde']

f_vasket = pd.concat( [ f_egenskap, f_veg ])

In [11]:
f_vasket.groupby([ 'vegkategori', 'trafikantgruppe']).agg( { 'Lengde' : 'sum' } )

Lengde
vegkategori trafikantgruppe         
E           K                789.000
R           G                522.098
            K                870.500

In [12]:
f_vasket['Lengde'].sum()

2181.598

In [15]:
f_telling = ff.drop_duplicates( ['nvdbId']).copy()

In [18]:
f_telling[ f_telling['vref'].str.contains( 'RV5')]

,objekttype,nvdbId,versjon,startdato,veglenkesekvensid,detaljnivå,typeVeg,kommune,fylke,vref,...,Lengde,Høyde,Gjennomsnittlig avstand fra veg,Adkomst,Etableringsår,Produsent,Eier,Vedlikeholdsansvarlig,Styrke,Helningsvinkel gjerde
12,845,916900165,1,2019-02-22,1201930,Vegtrase og kjørebane,Enkel bilveg,4640,46,RV5 S4D1 m948-1022,...,106.0,NaN,114.0,Helikopter,NaN,NaN,NaN,NaN,NaN,NaN
14,845,916900172,1,2019-02-22,1201930,Vegtrase og kjørebane,Enkel bilveg,4640,46,RV5 S4D1 m931-1086,...,221.0,3.0,NaN,Til fots,NaN,NaN,NaN,NaN,NaN,90.0
21,845,916900194,1,2019-02-22,1797330,Vegtrase og kjørebane,Enkel bilveg,4647,46,RV5 S12D1 m1661-1722,...,60.0,NaN,18.0,Kran,NaN,NaN,NaN,NaN,NaN,NaN
22,845,935936951,1,2019-06-17,384046,Vegtrase og kjørebane,Enkel bilveg,4602,46,RV5 S18D1 m3843-3891,...,50.0,5.0,NaN,NaN,2017.0,Geobrügg,NaN,NaN,NaN,NaN
23,845,935936952,1,2019-06-17,384046,Vegtrase og kjørebane,Enkel bilveg,4602,46,RV5 S18D1 m3739-3845,...,110.0,4.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,845,1004499759,2,2019-11-12,384046,Vegtrase og kjørebane,Enkel bilveg,4602,46,RV5 S18D1 m3704-3826,...,110.0,3.0,20.0,Kran,NaN,NaN,NaN,NaN,NaN,70.0


In [19]:
ls ..

lastned-nvdbrapporter/  nvdbrapporter/  vegbilder/
nvdbapi-V3/             temp/           workinprogress/
